In [ ]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [ ]:
data = {
    'NBD ON': [0.51, 0.36],
    'NBD LW': [0.46, 0.28],
    'NDB PIR': [0.42, 0.29],
    'SM PIR': [0.83, 0.86],
    'SM SP': [0.91, 0.92],
    'LFQ': [1.12, 1.14],
}

In [ ]:
nb = len(list(data.keys()))
nb

In [ ]:
angle = np.deg2rad([i*360/nb for i in range(0, nb)])
np.rad2deg(angle)

In [ ]:
radius = 1
steps = 5
circle = [(radius*math.cos(i*steps*math.pi/180), radius*math.sin(i*steps*math.pi/180)) for i in range(0, 360//steps)]
circle.append(circle[0])
coordinates0 = [(v[0]*radius*math.cos(angle[i]), v[0]*radius*math.sin(angle[i])) for i, (k, v) in enumerate(data.items())]
coordinates0.append(coordinates0[0])
coordinates1 = [(v[1]*radius*math.cos(angle[i]), v[1]*radius*math.sin(angle[i])) for i, (k, v) in enumerate(data.items())]
coordinates1.append(coordinates1[0])
coordinates0, coordinates1

In [ ]:
dfC = pd.DataFrame({
    'x': [x for x, y in circle], 
    'y': [y for x, y in circle],
})
df0 = pd.DataFrame({
    'x': [x for x, y in coordinates0], 
    'y': [y for x, y in coordinates0],
})
df1 = pd.DataFrame({
    'x': [x for x, y in coordinates1], 
    'y': [y for x, y in coordinates1],
})

In [ ]:
def project(df, color):
    return alt.Chart(df).mark_line().encode(
        alt.Latitude('x:Q'),
        alt.Longitude('y:Q'),
        color=alt.value(color),
    ).project(type="azimuthalEquidistant", rotate=[0, 0, 90])

ci = project(dfC, 'black')
c0 = project(df0, 'green')
c1 = project(df1, 'red')
ci+c0+c1